# Alex's Capstone Project

### Cincinnati, OH, US Report

### Is the presence of Bars, Gas Stations, and Liquor Stores correlated to Crime?

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation


!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize


! pip install folium==0.5.0
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

     |████████████████████████████████| 112kB 7.4MB/s eta 0:00:01
Folium installed
Libraries imported.


## 1. Pull Bar/Gas/Liquor Data from Foursquare

In [2]:
CLIENT_ID = 'TQQCN42M2IJGAI5B0EXF1TTOMMGVTOBJJGXCJZEXKY4DWXFX' # your Foursquare ID
CLIENT_SECRET = 'KJXXUVHENDUA1QSGIXJRUO5XGY2Y1FM1ZY55WCBTPYTYGPM1' # your Foursquare Secret
ACCESS_TOKEN = '0QD3WTYLBKT1OHQSBJGM35DHN4VL2YECHBFPR32SVAU30NUM' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 500
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: TQQCN42M2IJGAI5B0EXF1TTOMMGVTOBJJGXCJZEXKY4DWXFX
CLIENT_SECRET:KJXXUVHENDUA1QSGIXJRUO5XGY2Y1FM1ZY55WCBTPYTYGPM1


In [3]:
address = 'Cincinnati, OH'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

39.1014537 -84.5124602


In [4]:
#The first Foursquare query will be Bars within 30 miles of Cinci;
search_query1 = 'Bar'
radius = 50000
print(search_query1 + ' .... OK!')

#The second Foursquare query will be Gas Stations within 30 miles of Cinci;
search_query2 = 'Gas Station'
radius = 50000
print(search_query2 + ' .... OK!')

#The third Foursquare query will be Liquor Stores within 30 miles;
search_query3 = 'Liquor'
radius = 50000
print(search_query3 + ' .... OK!')

#Trying to find more liquor stores via query Spirits;
search_query4 = 'Spirits'
radius = 50000
print(search_query4 + ' .... OK!')

Bar .... OK!
Gas Station .... OK!
Liquor .... OK!
Spirits .... OK!


In [5]:
url1 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query1, radius, LIMIT)
url2 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query2, radius, LIMIT)
url3 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query3, radius, LIMIT)
url4 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query4, radius, LIMIT)

## 2. Keep only Relevant Info & Merge into One Frame

In [6]:
bars_json = requests.get(url1).json()
gas_json = requests.get(url2).json()
liquor_json = requests.get(url3).json()
spirits_json = requests.get(url4).json()

# assign relevant part of JSON to venues
venues1 = bars_json['response']['venues']
venues2 = gas_json['response']['venues']
venues3 = liquor_json['response']['venues']
venues4 = spirits_json['response']['venues']

# tranform venues into a dataframe
df_bars = json_normalize(venues1)
df_gas = json_normalize(venues2)
df_liquor = json_normalize(venues3)
df_spirits = json_normalize(venues4)

#only keep the relevant information
df_bars2 = df_bars[['name','location.postalCode','location.city']]
df_gas2 = df_gas[['name','location.postalCode','location.city']]
df_liquor2 = df_liquor[['name','location.postalCode','location.city']]
df_spirits2 = df_spirits[['name','location.postalCode','location.city']]

#rename columns
df_bars2.rename(columns={"name": "Venue", "location.postalCode": "Zip","location.city":"City"},inplace=True)
df_gas2.rename(columns={"name": "Venue", "location.postalCode": "Zip","location.city":"City"},inplace=True)
df_liquor2.rename(columns={"name": "Venue", "location.postalCode": "Zip","location.city":"City"},inplace=True)
df_spirits2.rename(columns={"name": "Venue", "location.postalCode": "Zip","location.city":"City"},inplace=True)

df_bars2.dropna(inplace=True)
df_gas2.dropna(inplace=True)
df_liquor2.dropna(inplace=True)
df_spirits2.dropna(inplace=True)

df_bars2['Type']="Bar"
df_gas2['Type']="Gas Station"
df_liquor2['Type']="Liquor Store"
df_spirits2['Type']="Liquor Store"


/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:13: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  del sys.path[0]
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:14: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:15: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  from ipykernel import kernelapp as app
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:16: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

In [7]:
df_venues = pd.concat([df_bars2,df_gas2,df_liquor2,df_spirits2])

In [8]:
df_venues.reset_index(drop=True,inplace=True)

In [9]:
#In some cases multiple Zip codes will refer to the same neighborhood;
df_venues['Zip'].unique()

array(['45202', '41071', '41011', '45219', '45209', '45203', '45206',
       '45247', '45245', '45230', '45220', '45069', '45248', '41076',
       '41048', '41042', '45223', '45211', '45213', '41051', '45242',
       '45244', '47001', '45231', '45150', '45005', '45030', '45014',
       '45249', '45140', '45238', '41073', '41017', '41075', '45255',
       '41015', '45241', '41030', '41018', '41016', '45243', '45227',
       '41094', '45015', '45039', '45102', '45040', '45036', '45152',
       '45044', '45056'], dtype=object)

## 3. Bring in Crime Scores (Area Vibes)

In [10]:
#The Area Vibes crime scores are on a scale of 1-13, with 1 being the lowest crime (rated A+) and 13 being the highest crime (rated F).
#Crime scores are separated by zip code for the Cincinati area.
df_crime = pd.read_csv('areavibescsv.csv')
df_crime.head()

,Zip,Letter,Crime,Population
0,45202,C+,7,2226
1,41001,A+,1,2637
2,41071,D+,10,15321
3,45219,F,13,16520
4,45209,C+,7,10214


### On the Area Vibes website, the Population and Letter Rating are provided for each Zip Code.

#### The Letter Ratings were converted to numbers based on ordinal scale.

## 4. Merge into Model Dataset and Handle Duplicate Zip Codes.

In [11]:
#First let's take a quick look at venues again;
df_venues.head()

,Venue,Zip,City,Type
0,Madonna's Bar & Grill,45202,Cincinnati,Bar
1,Westin Lobby Bar,45202,Cincinnati,Bar
2,Arnold's Bar & Grill,45202,Cincinnati,Bar
3,Bar 901,45202,Cincinnati,Bar
4,Cincinnati Bar Center,45202,Cincinnati,Bar


In [48]:
#I'm going to make 3 different dataframes with the counts of bars, gas stations, and liquor stores by zip code;
#This is the first one (bars only);
temp1 = df_venues.loc[df_venues['Type'] == 'Bar']
bars = temp1.groupby(['Zip']).count().drop(columns=['Venue','City']).reset_index()
bars[['Bars']] = bars[['Type']]
bars.drop(columns=['Type'],inplace=True)
bars
#Downtown Cinci has almost all the bars in the entire dataset;

,Zip,Bars
0,41011,4
1,41071,3
2,45202,33
3,45219,2


In [52]:
#Same situation for gas stations;
temp2 = df_venues.loc[df_venues['Type'] == 'Gas Station']
stations = temp2.groupby(['Zip']).count().drop(columns=['Venue','City']).reset_index()
stations[['Gas Stations']] = stations[['Type']]
stations.drop(columns=['Type'],inplace=True)
stations.head(5)
#The gas stations are much more evenly dispersed throughout the city;

,Zip,Gas Stations
0,41011,2
1,41042,3
2,41048,1
3,41051,1
4,41076,2


In [54]:
#Same situation for liquor stores;
temp3 = df_venues.loc[df_venues['Type'] == 'Liquor Store']
liquors = temp3.groupby(['Zip']).count().drop(columns=['Venue','City']).reset_index()
liquors[['Liquor Stores']] = liquors[['Type']]
liquors.drop(columns=['Type'],inplace=True)
liquors.head(5)
#The liquor stores are again pretty evenly distributed;

,Zip,Liquor Stores
0,41011,12
1,41015,4
2,41016,1
3,41017,6
4,41018,3


In [56]:
#To merge them all, we need to do outer joins on Zip.  Then any missing data will mean that the count of that variable was zero.
bars_and_stations = pd.merge(bars,stations,on='Zip',how='outer')

In [62]:
#The order of the merges does not matter here.  First I merge bars and gas stations, then merge liquor stores after;
final_venues = pd.merge(bars_and_stations,liquors,on='Zip',how='outer').fillna(0)

In [63]:
#After replacing all NaNs with zero, we have accurate counts for each zip code;
final_venues.head(10)

,Zip,Bars,Gas Stations,Liquor Stores
0,41011,4.0,2.0,12.0
1,41071,3.0,0.0,9.0
2,45202,33.0,0.0,4.0
3,45219,2.0,0.0,1.0
4,41042,0.0,3.0,2.0
5,41048,0.0,1.0,1.0
6,41051,0.0,1.0,2.0
7,41076,0.0,2.0,5.0
8,45005,0.0,1.0,0.0
9,45014,0.0,2.0,0.0


In [70]:
#Now let's merge the venue data into the crime data;
final_venues['Zip']=final_venues['Zip'].astype(int)
df_crime['Zip']=df_crime['Zip'].astype(int)
final_data = pd.merge(df_crime,final_venues,on='Zip',how='inner')

In [71]:
final_data

,Zip,Letter,Crime,Population,Bars,Gas Stations,Liquor Stores
0,45202,C+,7,2226,33.0,0.0,4.0
1,41071,D+,10,15321,3.0,0.0,9.0
2,45219,F,13,16520,2.0,0.0,1.0
3,45209,C+,7,10214,0.0,1.0,3.0
4,45206,F,13,298011,0.0,2.0,0.0
5,45203,C+,7,3996,0.0,1.0,0.0
6,45247,B-,6,19068,0.0,2.0,0.0
7,45245,B+,4,7087,0.0,2.0,0.0
8,45230,F,13,11430,0.0,1.0,0.0
9,45220,C+,7,8188,0.0,1.0,0.0


## 5. Visualization, Model, and Conclusion

In [75]:
#Let's look at the total number of each venue by crime rating;
final_data.groupby('Crime').sum().reset_index()

,Crime,Zip,Population,Bars,Gas Stations,Liquor Stores
0,1,870336,220252,0.0,8.0,40.0
1,2,88049,9792,0.0,2.0,1.0
2,4,90395,19728,0.0,3.0,0.0
3,5,90255,56422,0.0,2.0,1.0
4,6,90283,39604,0.0,2.0,1.0
5,7,225890,46565,33.0,3.0,9.0
6,10,123188,62377,3.0,3.0,13.0
7,12,90049,97054,0.0,1.0,1.0
8,13,493051,1368525,2.0,12.0,7.0


In [82]:
#Looks like I'm going to come to the opposite conclusion of what my hypothesis was?
#Let's do a model just to see;
from sklearn.preprocessing import PolynomialFeatures
from sklearn import linear_model
from sklearn import svm
model_data = final_data[['Crime','Bars','Gas Stations','Liquor Stores']]
model_x = model_data[['Bars','Gas Stations','Liquor Stores']]
model_y = model_data[['Crime']]


In [83]:
poly = PolynomialFeatures(degree=3)
train_x_poly = poly.fit_transform(model_x)
train_x_poly.shape

(49, 20)

In [84]:
clf = svm.SVC(kernel='rbf')
clf.fit(train_x_poly, model_y) 

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [90]:
clf.score(train_x_poly,model_y)

0.5714285714285714

In [92]:
#The mean accuracy is not good - we conclude that vicinity of bars, gas stations, and liquor stores does not affect crime;
#Let's do a linear regression as well so we can interpret the coefficients and t-values;
linmodel = linear_model.LinearRegression()
linmodel.fit(model_x,model_y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [94]:
linmodel.score(model_x,model_y)

0.09372625344027286

In [95]:
#The r-squared is only 9% which shows that the number of gas stations, liquor stores, and bars does not affect crime at all;
import statsmodels.api as sm

In [96]:
X2 = sm.add_constant(model_x)
est = sm.OLS(model_y, X2)
est2 = est.fit()
print(est2.summary())

                            OLS Regression Results                            
Dep. Variable:                  Crime   R-squared:                       0.094
Model:                            OLS   Adj. R-squared:                  0.033
Method:                 Least Squares   F-statistic:                     1.551
Date:                Mon, 21 Dec 2020   Prob (F-statistic):              0.214
Time:                        19:19:34   Log-Likelihood:                -145.57
No. Observations:                  49   AIC:                             299.1
Df Residuals:                      45   BIC:                             306.7
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const             5.2067      1.223      4.258

In [ ]:
#The variable closest to statistical significance is Gas Stations.  But it is still not significant;
#Actually, Liquor Stores show a negative correlation with crime;  That is interesting as well;